In [1]:
import hail as hl
from hail.plot import show
from pprint import pprint
hl.init(app_name="qc_plot_testing.ipynb")
hl.plot.output_notebook()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eriurn/python37_intel/lib/python3.7/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.0.0-78/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-01-24 09:53:54 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-01-24 09:53:55 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.1.2
SparkUI available at http://dice01-mlx.mlx:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.68-13190f0b6103
LOGGING: writing to /home/eriurn/notebooks/hail-20220124-0953-0.2.68-13190f0b6103.log


Loading BokehJS ...

In [2]:
hl.utils.get_1kg('data/')

2022-01-24 09:54:38 Hail: INFO: 1KG files found


In [3]:
mt = hl.read_matrix_table('data/1kg.mt')
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    }
----------------------------------------
Entry fields:
    'GT': call
    'AD': array<int32>
    'DP': int32
    'GQ': int32
    'PL': array<int32>
----------------------------------------
Colu

In [6]:
sample_qc_mt = hl.sample_qc(mt)

In [7]:
sample_qc_mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'sample_qc': struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
        n_transversion: int64, 
        n_star: int64, 
        r_ti_tv: float64, 
        r_het_hom_var: float64, 
        r_insertion_deletion: float64
    }
----------------------------

In [ ]:
original_DP = hl.plot.histogram(mt.DP, range=(0,30), bins=30, title="Original DP Histogram", legend="DP")
show(original_DP)

In [ ]:
sample_qc_call_rate_plot = hl.plot.histogram(sample_qc_mt.sample_qc.call_rate, range=(0.88,1), legend="Call Rate")
show(sample_qc_call_rate_plot)

In [ ]:
imputed_sex_mt = hl.impute_sex(mt.GT)

In [ ]:
imputed_sex_mt.describe()

In [ ]:
imputed_sex_mt.show()

In [ ]:
# Scatter plot of the observed homozygosity against the expected homozygosity. Points are coloured based on gender.
# Generally, males present higher observed homozygosity relative to females.
# NOTE: Currently unable to get the NA values to show still
is_female_plot = hl.plot.scatter(imputed_sex_mt.expected_homs, imputed_sex_mt.observed_homs, xlabel="Expected Homozygous", ylabel="Observed Homozygous", label=imputed_sex_mt.is_female, title="Observed Homozygosity vs Expected Homozygosity")
show(is_female_plot)

In [ ]:
# When provided the label of is_female, NA values are ignored
exp_vs_obs_plot = hl.plot.scatter(imputed_sex_mt.expected_homs, imputed_sex_mt.observed_homs, xlabel="Expected Homozygous", ylabel="Observed Homozygous", )
show(exp_vs_obs_plot)